# **Segmenting and Clustering Neighborhoods in Toronto**

#### This notebook will contain all three parts of the peer-graded assignment for the third week of the course Applied Data Science Capstone, part of the IBM Data Science certification on Coursera.

# Part 1 - Web Scraping

#### Import the libraries, in principle those relevant to this first part of the assignment.

In [1]:
import pandas as pd
import numpy as np

#### Scrape the Wikipedia page given in the instructions

In [2]:
# Use the Pandas method .read_html()
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Wtable = pd.read_html(url)

Wtable will be a list of a number of tables. In this case, three. The first one is the one of interest for this assignment.

In [3]:
# Access the table of interest
Wtable[0].head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


To start cleaning the table of interest, this will be put under another name.

In [4]:
# Re-assign the table of interest.
df_Toronto = Wtable[0]
display(df_Toronto.head())
df_Toronto.shape

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


(180, 3)

This table, or dataframe, already has the requiered column names. It shows, though, rows containing the string 'Not assigned' in the Borough column. These rows will be dropped.

In [5]:
df_Toronto.drop(df_Toronto[df_Toronto['Borough'] == 'Not assigned'].index, inplace = True)

It will be checked that neither the column Borough nor the Neighbouhood one have rows containing the string 'Not assigned'.

In [6]:
df_Toronto[df_Toronto['Borough'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


In [7]:
df_Toronto[df_Toronto['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


Both results displayed were empty, so no row contains the string 'Not assigned' in its columns. 

It will be checked if the column 'Postal Code' contains dupicated items.

In [8]:
print('Number of rows in the dataframe is:', df_Toronto.index.shape)
print('Number of unique elements in \'Postal Code\' is:', df_Toronto['Postal Code'].unique().shape)

Number of rows in the dataframe is: (103,)
Number of unique elements in 'Postal Code' is: (103,)


From this it is seen that no postal code appears more than once in the dataframe.

Because of the cleaning, many indices were lost, leaving gaps in their numbers. Then, the indices will be reseted.

In [9]:
df_Toronto.reset_index(inplace = True)
df_Toronto.drop('index', axis=1, inplace=True) # Drop the duplicate index column.

Finally for this part of the assignment, the shape of the dataframe will be checked after showing how its cleaning resulted.

In [10]:
display(df_Toronto.head(10))
display(df_Toronto.shape)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


(103, 3)

#### This concludes part 1 of this peer-graded assignment.

# Part 2 - Getting the coordinates

In the first place, an attempt was made of using geocoder, but it failed epically. It kept making calls until the allowed 2500 were reached and no coordinates were obtained. Because of that, it was decided to continue using the geospatial data provided in the instructions, in a csv file.

This file will be downloaded:

In [11]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('The data has been downloaded!')

The data has been downloaded!


Now, the data in the csv file will be read into a dataframe object.

In [12]:
GSdata = pd.read_csv('Geospatial_Coordinates.csv')
print('The data has been read into a dataframe')

The data has been read into a dataframe


A first exploration of how this dataframe looks and its shape will be done:

In [13]:
display(GSdata.head())
print('The geospatial dataframe has the shape: ', GSdata.shape)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


The geospatial dataframe has the shape:  (103, 3)


The number of rows matches that of the df_Toronto dataframe created in Part 1. For further certainty, it will be checked if all the elements in column 'Postal Code' are unique elements.

In [14]:
GSdata['Postal Code'].unique().shape

(103,)

Once again, the matching shape indicates all rows in the GSdata dataframe, in its column 'Postal Code' have unique elements.

It is then assumed that all the postal codes in GSdata and df_Toronto do not repeat in each dataframe and are the same in both dataframes.

In the df_Toronto dataframe two columns will be created. They will contain only np.nan elements to start with, and they will be named 'Latitude' and 'Longitude'.

In [15]:
df_Toronto['Latitude'] = np.nan
df_Toronto['Longitude'] = np.nan

It will be briefly checked if this was correctly done.

In [16]:
df_Toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN


Now, using two loops and a condition, the 'Postal Code' columns in both dataframes will be swept. In doing so, each element in the column in the df_Toronto database will be compared to all the elements in the column in the GSdata dataframe. When both elements are equal, the values of 'Latitude' and 'Longitude' will be copied from the GSdata dataframe to the df_Toronto dataframe, always minding to use the correct row numbers.

In [17]:
for i in range(len(df_Toronto['Postal Code'])):
    for j in range(len(df_Toronto['Postal Code'])):
        if df_Toronto['Postal Code'][i] == GSdata['Postal Code'][j]:
            df_Toronto['Latitude'][i] = GSdata['Latitude'][j]
            df_Toronto['Longitude'][i] = GSdata['Longitude'][j]

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


A check will be carried out to corroborate that all postal codes matched and no row in the df_Toronto database was left without coordinates.

In [18]:
df_Toronto[df_Toronto['Latitude'] == np.nan]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


In [19]:
df_Toronto[df_Toronto['Longitude'] == np.nan]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


After having verified this, the resulting dataframe will be shown and the types of data in its columns.

In [20]:
display(df_Toronto.head(10))
display(df_Toronto.dtypes)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Postal Code       object
Borough           object
Neighbourhood     object
Latitude         float64
Longitude        float64
dtype: object

#### This concludes part 2 of this peer-graded assignment.

# Part 3 - Analysis Toronto

### Exploration of the dataset.

#### In this part it will be undertaken the same kind of proceeding as in the lab "Segmenting and Clustering Neighborhoods in New York City".

The first task will be importing relevant dependencies.

In [21]:
pd.set_option('display.max_columns', None)          # I'm not convinced I need these options. We'll see.
pd.set_option('display.max_rows', None)             # I'm not convinced I need these options. We'll see.
import json                                         # library to handle JSON files
#!conda install -c conda-forge geopy --yes          # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim               # convert an address into latitude and longitude values
import requests                                     # library to handle requests
from pandas.io.json import json_normalize           # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm                          # Matplotlib and associated plotting modules
import matplotlib.colors as colors                  # Matplotlib and associated plotting modules
from sklearn.cluster import KMeans                  # import k-means from clustering stage
#!conda install -c conda-forge folium=0.5.0 --yes   # uncomment this line if you haven't completed the Foursquare API lab
import folium                                       # map rendering library

print('The libraries have been imported!')

The libraries have been imported!


Now, the Geopy library can be used to get the coordinates of the city of Toronto, which will be useful to create a map of the city. The instance of the geocoder will be named *to_explorer* (because I'm not that original).

In [22]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = "to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}°, {}°.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817°, -79.3839347°.


A map of the City of Toronto will be created and markers for the neighbourhoods will be added onto it. To be precise, each marker will symbolize the center of a postal code area. Clicking on them one can see the pop-up label and, many times, it will show the name of more that one neighbourhood, but only one postal code.

In [23]:
# Creation of the Toronto map, using latitude and longitude values.
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

# Addition of markers to the map.
for pcode, lat, lng, borough, neighbourhood in zip(
    df_Toronto['Postal Code'], 
    df_Toronto['Latitude'], 
    df_Toronto['Longitude'], 
    df_Toronto['Borough'], 
    df_Toronto['Neighbourhood']):
        label = '{}, {}, {}'.format(neighbourhood, borough, pcode)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
    
map_toronto

It is possible to see how many neighbourhoods there are in each borough, as follows.

In [24]:
df_Toronto.groupby(['Borough'])['Neighbourhood'].count()

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighbourhood, dtype: int64

Similarly to what was done in the lab abouth NYC, for simplicity a subselection of boroughs will be done. In this case, as the instructions suggest, all boroughs which have the word 'Toronto', and all their neighbourhoods, will be selected.

In [25]:
TorontoSub = df_Toronto[df_Toronto.Borough.str.contains('Toronto')].reset_index(drop = True)
display(TorontoSub.head(10))
TorontoSub.shape

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


(39, 5)

This shows there are a total of 39 neighbourhoods in this dataframe. It is also possible to see how many neighbourhoods has each borough in this subselection.

In [26]:
TorontoSub.groupby(['Borough'])['Neighbourhood'].count()

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighbourhood, dtype: int64

A dataframe is created with the coordinates of each borough in the subselection, which will be obtained as the mean coordinates among the ones of each neighbourhood in said borough.

In [27]:
TorontoCoor = TorontoSub.groupby(['Borough'])['Latitude', 'Longitude'].mean()
TorontoCoor.reset_index(inplace = True)
TorontoCoor

,Borough,Latitude,Longitude
0,Central Toronto,43.701980,-79.398954
1,Downtown Toronto,43.654597,-79.383972
2,East Toronto,43.669436,-79.324654
3,West Toronto,43.652653,-79.449290


Analogously, from the coordinates recently obtained for each borough in the subselection, the mean coordinates will be extracted. This will be useful to center a map showing markers for the neighbourhoods in the subselection, but also different markers for an estimated geographical center of each borough.

In [28]:
# Calculate the mean coordinates among the four subselected boroughs.
coordTorontoBoroughs = TorontoCoor[['Latitude', 'Longitude']].mean()
coordTorontoBoroughs

Latitude     43.669667
Longitude   -79.389217
dtype: float64

Having properly prepared the data, the new map can be created.

In [29]:
# Creation of the Toronto subselection map, using latitude and longitude values. If this cell is run, then
# all the following pertaining to the markers will have to be run, too, not only the one displaying the map.
map_torontoSub = folium.Map(location = [coordTorontoBoroughs[0], coordTorontoBoroughs[1]], zoom_start = 12)

Following, the markers for each neighbourhood in the subselection.

In [30]:
# Addition of markers to the map.
for pcode, lat, lng, borough, neighbourhood in zip(
    TorontoSub['Postal Code'], 
    TorontoSub['Latitude'], 
    TorontoSub['Longitude'], 
    TorontoSub['Borough'], 
    TorontoSub['Neighbourhood']):
        label = '{}, {}, {}'.format(neighbourhood, borough, pcode)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=4,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_torontoSub)  

Now, the following markers are those corresponding to each estimated borough center.

In [31]:
# Addition of markers to the map.
for borough, lat, lng in zip(
    TorontoCoor['Borough'], 
    TorontoCoor['Latitude'], 
    TorontoCoor['Longitude']):
        label = '{}'.format(borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=8,
            popup=label,
            color='orange',
            fill=True,
            fill_color='#ffae1a',
            fill_opacity=0.3,
            parse_html=False).add_to(map_torontoSub)

Plot the map.

In [32]:
map_torontoSub

It is good to remember that the marker of each neighbourhood in the subselection represents the center of a postal code area, which can be actually comprised of more than one neighbourhood.

### Exploration the neighbourhoods in the Toronto subselection.

In the following cell the credentials for calling Foursquare will be defined. To do this, it is necessary a Client ID and a Client secret. Given that those are personal information, sensitive, the following cell will be hidden.

In [33]:
# @hidden_cell
CLIENT_ID = 'TYVQKMG0CUTNR3GVI2GKJSVUO4GCKTSQD1TEUEP4CZLJML3F'      # your Foursquare ID
CLIENT_SECRET = 'ACZ1WCYNFDWBOTGW43BM1GWPINVJZGHGWLX2Z0CEXKUQQRU0'  # your Foursquare Secret

Also, for calling Foursquare, it is necessary to define the version and a limit value.

In [34]:
VERSION = '20180605'                                                # Foursquare API version
LIMIT = 100                                                         # A default Foursquare API limit value

A function from lab **"Segmenting and Clustering Neighborhoods in New York City"** will be borrowed. This will allow the repeated extraction of the category of the venue, along with its name and coordinates, for all the neighborhoods in the Toronto subselection.

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neigh. Latitude', 
                  'Neigh. Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now, the borrowed function will be used:

In [36]:
TorontoSub_venues = getNearbyVenues(names = TorontoSub['Neighbourhood'],
                                    latitudes = TorontoSub['Latitude'],
                                    longitudes = TorontoSub['Longitude']
                                    )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

Corroboration of the shape and of the first rows in the created dataframe.

In [37]:
print(TorontoSub_venues.shape)
TorontoSub_venues.head()

(1624, 7)


,Neighbourhood,Neigh. Latitude,Neigh. Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


Amount of venues by neighbourhood.

In [38]:
TorontoSub_venues.groupby('Neighbourhood', as_index = False)['Venue'].agg({'Count': 'count'})

,Neighbourhood,Count
0,Berczy Park,55
1,"Brockton, Parkdale Village, Exhibition Place",23
2,"Business reply mail Processing Centre, South C...",16
3,"CN Tower, King and Spadina, Railway Lands, Har...",16
4,Central Bay Street,68
5,Christie,16
6,Church and Wellesley,75
7,"Commerce Court, Victoria Hotel",100
8,Davisville,33
9,Davisville North,9


See how many unique venue categories there are.

In [39]:
print('There are {} uniques venue categories.'.format(len(TorontoSub_venues['Venue Category'].unique())))

There are 236 uniques venue categories.


### Analysis of each neighbourhood.

The data set will be prepared once again so that it's adecuate to be used with a k-Means clustering model. Each kind of venue will be converted in a column. When a neighbourhood presents this venue, a one will be shown at the intersection of the neighbourhood row and the venue column.

In [40]:
# One hot encoding
TorontoSub_onehot = pd.get_dummies(TorontoSub_venues[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe.
TorontoSub_onehot['Neighbourhood'] = TorontoSub_venues['Neighbourhood'] 

# Move neighbourhood column to the first place.
fixed_columns = [TorontoSub_onehot.columns[-1]] + list(TorontoSub_onehot.columns[:-1])
TorontoSub_onehot = TorontoSub_onehot[fixed_columns]

display(TorontoSub_onehot.head())
TorontoSub_onehot.shape

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

(1624, 237)

Next, rows will be grouped by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [41]:
TorontoSub_grouped = TorontoSub_onehot.groupby('Neighbourhood').mean().reset_index()
TorontoSub_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.018182,0.036364,0.000000,0.000000,0.00,0.018182,0.018182,0.000000,0.036364,0.000000,0.000000,0.018182,0.0000,0.000000,0.0000,0.00,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.018182,0.00,0.000000,0.0000,0.000000,0.036364,0.000000,0.000000,0.000000,0.000000,0.000000,0.036364,0.090909,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.018182,0.000000,0.018182,0.000000,0.000000,0.000000,0.018182,0

In [42]:
TorontoSub_grouped.shape

(39, 237)

The shape of this new dataframe checks in the number of rows, meaning each one corresponds to one neighbourhood (more precisely, one postal code).

This is a function that sorts venues according to their occurence frequency.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

A new dataframe will be created, showing the most common venues in each neighbourhood.

In [44]:
num_top_venues = 10 # This sets how many venues per neighbourhood will be displayed.

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
TorNeigh_venues_sorted = pd.DataFrame(columns=columns)
TorNeigh_venues_sorted['Neighbourhood'] = TorontoSub_grouped['Neighbourhood']

for ind in np.arange(TorontoSub_grouped.shape[0]):
    TorNeigh_venues_sorted.iloc[ind, 1:] = return_most_common_venues(TorontoSub_grouped.iloc[ind, :], num_top_venues)

Neighbourhood by neighbourhood it can be seen what venues are the most common in each of them.

In [45]:
TorNeigh_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Bakery,Sandwich Place,Department Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Climbing Gym,Burrito Place,Restaurant,Italian Restaurant,Intersection,Bar
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Light Rail Station,Skate Park,Burrito Place,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Harbor / Marina,Boat or Ferry,Rental Car Location,Bar,Plane,Coffee Shop,Sculpture Garden
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Department Store,Burger Joint,Japanese Restaurant,Thai Restaurant
5,Christie,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Nightclub,Baby Store,Restaurant,Athletics & Sports,Coffee Shop
6,Church and Wellesley,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant,Pub,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Japanese Restaurant,Cocktail Bar
8,Davisville,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Gym,Toy / Game Store,Greek Restaurant
9,Davisville North,Park,Hotel,Breakfast Spot,Sandwich Place,Food & Drink Shop,Dog Run,Dance Studio,Department Store,Gym / Fitness Center,Cosmetics Shop


### Clustering of the subselected neighbourhoods.

The model k-Means will be used to cluster the neighbourhoods. **As a clarifying remark**, this model will attempt to find similarities among neighbourhoods related to the venues that are more frequent in them. Nothing to do will have any geographic characteristic about them.

In [46]:
# Set the number of clusters.
kclusters = 5

# Dropping the column Neighbourhood leaves the dataset unlabeled, which is essential for k-Means clustering.
TorontoSub_grouped_clustering = TorontoSub_grouped.drop('Neighbourhood', 1)

# Run k-means clustering.
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(TorontoSub_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 2, 1,
       1, 1, 1, 1, 0, 3, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1], dtype=int32)

New dataframe that includes the cluster as well as the top ten venues for each neighborhood.

In [47]:

TorNeigh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) # Adds clustering labels.

TorontoMerg = TorontoSub

# Merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
TorontoMerg = TorontoMerg.join(TorNeigh_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

TorontoMerg.head() 

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Electronics Store,Spa,Beer Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Yoga Studio,Diner,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,Gastropub,American Restaurant,Farmers Market,Hotel,Japanese Restaurant
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


Now it is possible to visualize the obtained k-Means clustering. It's good to remember that the coordinates of the city of Toronto are stored in the variables 'latitude' and 'longitude'.

In [56]:
# Create map. If this cell is run, all the following related to this map need to be run, not only the one displaying it.
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

The next cell will add the markers onto the map, while chosing a color for them.

In [57]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(TorontoMerg['Latitude'], 
                                  TorontoMerg['Longitude'], 
                                  TorontoMerg['Neighbourhood'], 
                                  TorontoMerg['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

Lastly, plot the map.

In [58]:
map_clusters

### Examination of the obtained clusters.

Slices of the dataframe containing all the neighbourhoods with their cluster labels and their most common venues will be shown. In particular, one slice for each cluster label, or, in other words, showing one cluster at a time. This is done calling the column of the neighbourhood name [2] and then a list of columns that begin with the one of the cluster label and continues with all the ones of the most common venues.

The clusters have been named mostly related to the first columns of most common venues.

#### First cluster - Promenade cluster.

In [51]:
TorontoMerg.loc[TorontoMerg['Cluster Labels'] == 0, TorontoMerg.columns[[2] + list(range(5, TorontoMerg.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,0,Park,Swim School,Bus Line,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
21,"Forest Hill North & West, Forest Hill Road Park",0,Park,Sushi Restaurant,Jewelry Store,Trail,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
33,Rosedale,0,Park,Trail,Playground,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Second cluster - Toronto cluster.

In [52]:
TorontoMerg.loc[TorontoMerg['Cluster Labels'] == 1, TorontoMerg.columns[[2] + list(range(5, TorontoMerg.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",1,Coffee Shop,Park,Pub,Bakery,Theater,Breakfast Spot,Café,Electronics Store,Spa,Beer Store
1,"Queen's Park, Ontario Provincial Government",1,Coffee Shop,Yoga Studio,Diner,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop
2,"Garden District, Ryerson",1,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant
3,St. James Town,1,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,Gastropub,American Restaurant,Farmers Market,Hotel,Japanese Restaurant
5,Berczy Park,1,Coffee Shop,Seafood Restaurant,Cocktail Bar,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Bakery,Sandwich Place,Department Store
6,Central Bay Street,1,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bubble Tea Shop,Department Store,Burger Joint,Japanese Restaurant,Thai Restaurant
7,Christie,1,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Nightclub,Baby Store,Restaurant,Athletics & Sports,Coffee Shop
8,"Richmond, Adelaide, King",1,Coffee Shop,Café,Hotel,Gym,Restaurant,Clothing Store,Thai Restaurant,Bar,Bookstore,Burrito Place
9,"Dufferin, Dovercourt Village",1,Pharmacy,Bakery,Park,Middle Eastern Restaurant,Bank,Bar,Café,Supermarket,Music Venue,Brewery
10,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Fried Chicken Joint,Brewery,Scenic Lookout,Bar,Music Venue


#### Third cluster - Kids cluster.

In [53]:
TorontoMerg.loc[TorontoMerg['Cluster Labels'] == 2, TorontoMerg.columns[[2] + list(range(5, TorontoMerg.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",2,Playground,Trail,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Fourth cluster - Music cluster.

In [54]:
TorontoMerg.loc[TorontoMerg['Cluster Labels'] == 3, TorontoMerg.columns[[2] + list(range(5, TorontoMerg.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,3,Music Venue,Garden,Yoga Studio,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


#### Fifth cluster - Housing cluster.

In [55]:
TorontoMerg.loc[TorontoMerg['Cluster Labels'] == 4, TorontoMerg.columns[[2] + list(range(5, TorontoMerg.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,The Beaches,4,Neighborhood,Pub,Health Food Store,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant


From the clusters obtained, it can be thought that a lot of this study is about a downtown area, presenting a cluster that clearly predominates.

0. (Label = **0**) The **Promenade cluster** is the first one listed, but second in size. Clearly there's a lot of green in it and other outdoors activities for kids and adults. 
1. (Label = **1**) If one examines the **Toronto cluster**, which is the second listed, it will be noticed that a great number of the most common venues are reastaurants, pubs and coffee shops. This cluster if by far the largest, which seems to make sense, considering that Toronto is a big city.
2. (Label = **2**) From now on, the listed clusters seem to be outlier neighbourhoods, given that each cluster contains only one neighbourhood. In particular, this is the **kids cluster**, as the most common venues are playgrounds.
3. (Label = **3**) The **music cluster** appears to be outstanding in the musical offer.
4. (Label = **4**) Finally, the **housing cluster** seems to be the one mostly dedicated to residential areas.